In [77]:
import Pkg
Pkg.activate("MMLDS_project")
using DynamicalSystems, ReservoirComputing, Plots, Printf
using DifferentialEquations
using OrdinaryDiffEq

  Activating project at `c:\Users\tomfi\Desktop\Uni\3M_semester\Project_MMLDS\MMLDS_2\MMLDS_project\src\MMLDS_project`


In [78]:
#Pkg.add("NetCDF")
using NetCDF

In [79]:
filename = "sst.mon.mean.nc"

"sst.mon.mean.nc"

In [80]:
x = ncread(filename, "sst")

360×180×2040 Array{Float32, 3}:
[:, :, 1] =
 -1.707  -1.727  -1.74   -1.765  -1.76   …  1.0f20  1.0f20  1.0f20  1.0f20
 -1.707  -1.73   -1.742  -1.765  -1.76      1.0f20  1.0f20  1.0f20  1.0f20
 -1.707  -1.73   -1.747  -1.767  -1.762     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.732  -1.745  -1.77   -1.765     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.732  -1.747  -1.772  -1.772     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.735  -1.747  -1.777  -1.775  …  1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.735  -1.747  -1.772  -1.775     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.732  -1.75   -1.772  -1.772     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.732  -1.745  -1.772  -1.775     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.732  -1.747  -1.77   -1.777     1.0f20  1.0f20  1.0f20  1.0f20
  ⋮                                      ⋱                          
 -1.705  -1.707  -1.71   -1.735  -1.742     1.0f20  1.0f20  1.0f20  1.0f20
 -1.705  -1.707  -1.715  -1.745  -1.742     1.0f20  1.0f20  1.

In [81]:
x_reduced = x[121:170, 86:95, :] # es wird ab 1 gezählt
#120-170 Grad West können wir einfach so nehmen? (von wo wird im Datenset gezählt?)
#5 Grad vom Äquator aus gezählt ist genau in der Mitte, d.h. wir müssen die 10 Felder in der Mitte nehmen.
#180/2 = 90 -> 85-95

50×10×2040 Array{Float32, 3}:
[:, :, 1] =
 27.955  28.068  28.113  28.105  …   1.0f20   1.0f20  28.883   28.785
 27.933  27.998  28.023  28.015      1.0f20   1.0f20   1.0f20  28.743
 27.958  27.978  27.993  27.973     28.635   28.805   28.918    1.0f20
 27.99   27.975  28.023  28.018     28.665   28.845   28.923   28.838
 28.018  28.015  28.078  28.165     28.693   28.865   28.928   28.848
 27.995  28.053  28.135  28.225  …  28.738   28.898   28.93    28.858
 28.005  28.07   28.185  28.283     28.778   28.905   28.93    28.835
 28.138  28.19   28.203  28.315     28.773   28.903   28.915   28.81
 28.128  28.18   28.328  28.343     28.755   28.888   28.915   28.823
 28.125  28.203  28.335  28.46      28.748   28.888   28.94    28.845
  ⋮                              ⋱                             
 28.698  28.738  28.745  28.783     29.1     29.158   29.213   29.243
 28.675  28.703  28.688  28.688     29.04    29.118   29.175   29.21
 28.628  28.658  28.653  28.635     28.973   29.073   2

In [82]:
# are many more values missing?
x_reduced[1,10,:]

2040-element Vector{Float32}:
 28.785002
 28.538002
 28.578001
 29.205002
 29.038002
 28.368002
 27.435001
 27.170002
 27.533
 28.720001
  ⋮
 29.598001
 29.243002
 28.403002
 27.365002
 26.823002
 27.090002
 28.345001
 29.315
 30.153002

In [83]:

function sum_elements_without_empty_values(y)
    h,w = size(y)
    counter = 0
    summation = 0
    for i in 1:h
        for j in 1:w
            if y[i,j] != 1.0f20
                summation = summation + y[i,j]
                counter = counter + 1
            end
        end
    end
    return summation/counter #using the mean here
end

function sum_elements_without_empty_values_vec(y) #EMPTY VS ZERO
    bool_matrix = y .< 1.0f20 
    nonzero_elements = sum(bool_matrix)
    y_adjusted = y .* bool_matrix
    summation = sum(y_adjusted)
    return summation/nonzero_elements
end

function compress_data_matrix(x_data, kernel_size, vectorized = true)
    h,w,d = size(x_data)
    n_h = trunc(Int, h/kernel_size)
    n_w = trunc(Int, w/kernel_size)
    A = ones(n_h, n_w,d)
    for dim in 1:d
        for i in 1:n_h
            filter_region_h = (1 + (i-1)*kernel_size): (i*kernel_size)
            for j in 1:n_w
                filter_region_w = (1 + (j-1)*kernel_size): (j*kernel_size)
                y = x_reduced[filter_region_h, filter_region_w, dim]
                if vectorized
                    A[i,j,dim] = sum_elements_without_empty_values_vec(y)
                else
                    A[i,j,dim] = sum_elements_without_empty_values(y)
                end
            end
        end
    end
    return A
end

@btime compress_data_matrix(x_reduced, 5, true)
#print("Now the second approach")
@btime compress_data_matrix(x_reduced, 5, false)

  21.085 ms (356982 allocations: 21.32 MiB)


  13.892 ms (234582 allocations: 10.74 MiB)


10×2×2040 Array{Float64, 3}:
[:, :, 1] =
 28.0325  28.7257
 28.2426  28.7958
 28.5179  28.8938
 28.84    28.8633
 29.023   28.7771
 29.1572  29.029
 29.1264  29.3668
 28.9993  29.2807
 28.7191  29.0902
 28.4407  28.8026

[:, :, 2] =
 27.8742  28.5573
 28.2316  28.7403
 28.6131  28.9374
 28.8382  28.9983
 28.9616  28.9013
 29.1256  29.1433
 29.117   29.4022
 29.0324  29.2732
 28.7542  29.0125
 28.3725  28.662

[:, :, 3] =
 28.1076  28.5916
 28.488   28.7715
 28.8473  28.9322
 29.0742  29.0224
 29.1702  29.0346
 29.2598  29.2485
 29.196   29.4523
 29.076   29.3121
 28.6979  29.022
 28.2181  28.6716

;;; … 

[:, :, 2038] =
 29.4195  28.4254
 29.3835  28.0568
 29.723   28.7421
 30.2484  29.7091
 30.2277  29.9763
 30.4001  29.9691
 30.4189  30.424
 30.5088  30.6224
 30.6631  30.9208
 30.5643  30.8168

[:, :, 2039] =
 29.5172  29.3495
 29.6865  29.141
 29.7411  29.3249
 29.8994  29.5228
 29.8914  29.6906
 30.031   29.676
 30.0984  30.0788
 30.2152  30.2738
 30.3864  30.6285
 30.4019  30.6959

In [84]:
t1 = copy(x_reduced[1:5,1:5,1])

using BenchmarkTools

function sum_elements_without_zero_values(y)
    bool_matrix = y .< 1.0f20 
    nonzero_elements = sum(bool_matrix)
    summation = sum(y .* bool_matrix)
    return summation/nonzero_elements
end

function sum_elements_without_empty_values(y)
    h,w = size(y)
    counter = 0
    summation = 0
    for i in 1:h
        for j in 1:w
            if y[i,j] != 1.0f20
                summation = summation + y[i,j]
                counter = counter + 1
            end
        end
    end
    return summation/counter #using the mean here
end


@btime sum_elements_without_empty_values(t1)
@btime sum_elements_without_zero_values(t1)


  68.475 ns (1 allocation: 16 bytes)


  228.008 ns (4 allocations: 288 bytes)


28.03248f0